<a href="https://colab.research.google.com/github/BentonMiller/python1/blob/master/Copy_of_Quant_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Specify a path to your files here:**

 *A Sample Sheet must have the columns 'well_position', 'name', and 'concentration'.*

 *A name that begins with 'Std' [note the capitalization] will be used as a calibration curve data point.*

 *Only rows will a 'well_position' that match the XML row will be used.*




In [ ]:
data_file_path = '/content/drive/Shareddrives/Leash - Shared/Operations/Data/PCR Quant Data Files/M200 Results/2024-01-30 15-31-03_plate_1.xml'
sample_sheet_share_url = 'https://docs.google.com/spreadsheets/d/1SbBTsqcMe77MPC0eASFwYUtrT2cFauquXIbKYWnuOwg/edit?usp=drive_link'


**Connect to Google Drive and open files:**

In [ ]:
from google.colab import drive
from xml.dom import minidom
drive.mount('/content/drive')

wells = minidom.parse(data_file_path).getElementsByTagName("Well")
assert len(wells) == 385
data = []
for well in wells:
    well_position = well.getAttribute("Pos")
    measurement = list(well.getElementsByTagName("Single"))
    for single in measurement:
        value = float(single.firstChild.nodeValue)
        row = {'well_position':well_position, 'rfu_value':value}
        data.append(row)
assert len(data) == 384
print(len(data),' wells of data extracted from XML')

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open_by_url(sample_sheet_share_url).sheet1
sample_sheet = filter(None,worksheet.get_all_records())

sample_sheet_rows = []
for row in sample_sheet:
  sample_sheet_rows.append(row)
print(len(sample_sheet_rows),' sample sheet rows found:')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
384  wells of data extracted from XML
153  sample sheet rows found:


In [ ]:
import pandas as pd

sample_sheet_df = pd.DataFrame(sample_sheet_rows)
xml_df = pd.DataFrame(data)
data_df = xml_df.set_index('well_position').join(sample_sheet_df.set_index('well_position'))
data_df

,rfu_value,name,concentration
well_position,,,
A1,45731.0,std1000,1000
A2,43594.0,NaN,NaN
A3,25326.0,std500,500
A4,26675.0,NaN,NaN
A5,10297.0,std250,250
...,...,...,...
P5,7.0,NaN,NaN
P4,7.0,NaN,NaN
P3,7.0,NaN,NaN


**Calculate Concentrations:**

In [ ]:
from scipy import stats
pd.options.mode.chained_assignment = None # ignore warnings from pandas
standards = data_df[data_df['name'].str.contains('Std', na=False)]
background = data_df[data_df['name'].str.contains('Background', na=False)]
samples = data_df[~data_df['name'].str.contains('Std', na=False)]

x_range = standards['rfu_value']
y_range = standards['concentration']
background = background['rfu_value'].tolist()[0]
print(background,' background detected')

x_subrange = x_range.to_list()
y_subrange = y_range.to_list()
x_subrange_bg_subtract = [x-background for x in x_subrange]
slope,intercept,r,tt,stderr=stats.linregress(x_subrange_bg_subtract,y_subrange)

# calculate concentrations for samples here...

print("The linear equation for the fit is y = {0:1.4E} x + {1:1.4E}, with an R-squared value of {2:1.5f}.".format(slope, intercept, r**2))

samples['concentration'] = samples['rfu_value'].map(lambda rfu_value: slope*rfu_value+intercept )
samples['type'] = 'Sample'
standards['type'] = 'Standard'

7.0  background detected
The linear equation for the fit is y = 2.2948E-02 x + -1.9832E+00, with an R-squared value of 0.99901.


**Display Graph:**

In [ ]:
import plotly.express as px
# samples = samples.append(standards)
samples = pd.concat([samples,standards])
fig = px.scatter(samples,x="rfu_value",y="concentration", color='type', hover_data={"well_position": (samples.index)})
fig.show()
# samples.sort_values('well_position')

In [247]:
samples['row'] = samples.index
import re

samples['col'] = samples['row']
samples['col'] = samples['row'].map(lambda row: int(re.search('[0-9]+',row).group(0)))
samples['row'] = samples['row'].map(lambda row: re.search('[A-Z]+',row).group(0))

import string
samples['row_num'] = samples['row'].map(lambda row: string.ascii_uppercase.index(row)+1)


In [259]:
samples.pivot_table(index="row",columns="col",values="concentration").style.background_gradient()

col,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
row,,,,,,,,,,,,,,,,,,,,,,,,
A,1047.431750,998.392809,579.186735,610.142997,234.307817,241.054400,123.080981,122.461397,62.500000,62.155202,31.250000,31.130098,15.625000,17.338613,0.000000,-0.789962,-1.822602,-1.845550,-1.799655,-1.845550,-1.845550,-1.822602,-1.845550,-1.822602
B,-1.111228,-1.042385,-1.157123,-1.203018,-1.455441,-1.524284,-1.639022,-1.661969,-1.730812,-1.776707,-1.776707,-1.799655,-1.799655,-1.799655,-1.822602,-1.845550,-1.822602,-1.822602,-1.822602,-1.822602,-1.822602,-1.799655,-1.822602,-1.822602
C,1.940798,2.147326,2.606278,2.652173,7.104000,7.058105,7.815374,7.838322,4.855139,4.510925,3.661865,3.684813,2.353854,2.537435,2.055536,2.307959,1.940798,2.170274,3.455337,3.799551,1.458899,1.803113,2.560382,2.491540
D,-1.822602,-1.776707,-1.799655,-1.776707,-1.799655,-1.799655,-1.799655,-1.799655,-1.845550,-1.822602,-1.822602,-1.845550,-1.822602,-1.845550,-1.845550,-1.845550,-1.822602,-1.799655,-1.799655,-1.799655,-1.799655,-1.845550,-1.822602,-1.822602
E,4.923981,4.281450,5.428828,4.946929,8.297273,8.182535,9.926550,10.041288,2.698068,2.973439,2.904596,2.904596,7.081052,7.517056,5.910727,6.071359,10.844452,11.877093,6.300835,7.585899,3.042281,3.271757,2.307959,1.917851
F,-1.822602,-1.822602,-1.799655,-1.822602,-1.822602,-1.822602,-1.799655,-1.822602,-1.822602,-1.822602,-1.822602,-1.822602,-1.822602,-1.822602,-1.845550,-1.822602,-1.845550,-1.799655,-1.799655,-1.776707,-1.822602,-1.822602,-1.822602,-1.822602
G,1.642480,1.504795,4.510925,4.373240,4.671558,5.658303,9.215176,8.503801,0.173836,0.104993,1.711323,1.413004,2.124379,2.285012,3.179967,3.386495,2.009641,2.468592,2.353854,2.170274,-0.009745,0.288574,1.573637,1.114686
H,-1.822602,-1.822602,-1.845550,-1.799655,-1.822602,-1.822602,-1.822602,-1.822602,-1.868497,-1.845550,-1.845550,-1.822602,-1.822602,-1.822602,-1.822602,-1.845550,-1.822602,-1.822602,-1.822602,-1.822602,-1.822602,-1.822602,-1.822602,-1.822602
I,11.854145,12.909733,13.529317,14.057111,14.309534,15.250385,3.983131,4.740401,0.449207,0.540997,0.563944,0.518049,3.707761,4.717453,3.088176,3.386495,0.036151,0.196783,0.036151,-0.239220,2.812806,3.409442,-0.606381,-0.744067
